In [1]:
from keras.models import model_from_json
import keras
from keras.optimizers import Adam
import numpy as np
import copy
np.random.seed(0)
import pandas as pd
import ast
import os

from calculate_metrics import *

Using TensorFlow backend.


In [2]:
# Load model
with open("model.json") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

# Load weights
loaded_model.load_weights("model_weights.h5")
print("Model has been loaded")

Model has been loaded


In [3]:
loaded_model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

## Load the data

In [4]:
rescale_bool = True
# Load the data
print("Loading the data ...\n\n")
if rescale_bool:
    images = np.load("cropped_rescaled_images.npy")
    images_gt = np.load("cropped_rescaled_images_gt.npy")
else:
    images = np.load("cropped_images.npy")
    images_gt = np.load("cropped_images_gt.npy")

print("Image data has size: {}".format(images.shape))
print("Ground truth has size: {}".format(images_gt.shape))

# For now keep only the labels of the Left Venctricle (removes RV, myocardium)
images_gt[images_gt != 3] = 0
images_gt[images_gt == 3] = 1

# Zero-mean, unit-variance normalization
mean_per_slice = np.mean(images, axis=(1,2), keepdims=True)
std_per_slice =  np.std(images, axis=(1,2), keepdims=True)
images = (images - mean_per_slice) / std_per_slice

# Turn image data to 4D
images = np.reshape(images, newshape=(*images.shape, 1))
images_gt = np.reshape(images_gt, newshape=(*images_gt.shape, 1))

# Load patient information
patient_info = pd.read_csv("patient_info.csv", converters={"spacing": ast.literal_eval,"image_pixels": ast.literal_eval }) # the converter is used to convert back the tuple
id_list = patient_info["patient_id"].to_numpy()
image_sizes = patient_info["image_pixels"].to_numpy()
image_sizes = np.array([*image_sizes])
z_dim = image_sizes[:,2]
EF_gt = patient_info["ejection_fraction"].to_numpy()
spacing = patient_info["image_pixels"].to_numpy()
spacing = np.array([*spacing])

# Create an array of (1902,) where each row corresponds to the ID of every slice
# of the dataset
patient_id_array = np.array([])
for patient_id in id_list:
    patient_id_array = np.append(patient_id_array, np.full(shape=2*z_dim[patient_id-1], fill_value=patient_id))

print("The array of patient IDs has shape: ", patient_id_array.shape)

# Split dataset to train/valid/test based on patient ids (doesnt mix patient slices)
# and sample uniformly each of the 5 groups of patients
np.random.seed(0) # seed for reproducability
train_ids = np.array([], dtype=np.int32)
valid_ids = np.array([], dtype=np.int32)
test_ids = np.array([], dtype=np.int32)

# There are 5 patient groups with 20 patients each
samples_per_group = 20
num_of_groups = 5
for group in range(num_of_groups):
    # Shuffle the patients of each group and split to train/valid/test
    group_id_list = copy.deepcopy(id_list[group*samples_per_group:(group+1)*samples_per_group])
    np.random.shuffle(group_id_list)
    train_ids = np.append(train_ids, group_id_list[:14]) # 70% training
    valid_ids = np.append(valid_ids, group_id_list[14:17]) # 15% validation
    test_ids = np.append(test_ids, group_id_list[17:]) # 15% test
    
# Sort the split IDs (this is the order they are masked)
train_ids = copy.deepcopy(np.sort(train_ids))
valid_ids = copy.deepcopy(np.sort(valid_ids))
test_ids = copy.deepcopy(np.sort(test_ids))

# Create the id masks
train_msk = np.isin(patient_id_array, train_ids)
valid_msk = np.isin(patient_id_array, valid_ids)
test_msk = np.isin(patient_id_array, test_ids)

print("The train set consists of {} slices".format(np.count_nonzero(train_msk)))
print("The validation set consists of {} slices".format(np.count_nonzero(valid_msk)))
print("The test set consists of {} slices\n\n".format(np.count_nonzero(test_msk)))

# Now split the images based on the masks
train_set = images[train_msk]
train_set_gt = images_gt[train_msk]

valid_set = images[valid_msk]
valid_set_gt = images_gt[valid_msk]

test_set = images[test_msk]
test_set_gt = images_gt[test_msk]

Loading the data ...


Image data has size: (1902, 144, 144)
Ground truth has size: (1902, 144, 144)
The array of patient IDs has shape:  (1902,)
The train set consists of 1320 slices
The validation set consists of 264 slices
The test set consists of 318 slices




# Prediction on Validation Set

## Predict

In [5]:
# Predict on the validation set and save the prediction
if not(os.path.exists("prediction_scaled_crop_augmented.npy")):
    prediction = loaded_model.predict(valid_set, batch_size=10, verbose=1)
    np.save("prediction_scaled_crop_augmented", prediction)

In [6]:
# Load and preprocess the prediction
prediction = np.load("prediction_scaled_crop_augmented.npy")
prediction /= np.amax(prediction)
prediction = prediction[:,:,:,0]
print("The validation prediction has shape: ", prediction.shape)

valid_set = valid_set[:,:,:,0]
valid_set_gt = valid_set_gt[:,:,:,0]
valid_id_array = patient_id_array[valid_msk]

The validation prediction has shape:  (264, 144, 144)


## Dice score and Haussdorf distance

In [7]:
# Keep only predictions that equal to 1
validation_prediction = copy.deepcopy(prediction)
validation_prediction[validation_prediction<1] = 0

# Calculate dice score and haussdorf distance for each patient
dice_scores = []
haus_distances = []
index_start = 0
for patient_id in valid_ids:
    index_end = index_start + 2*z_dim[patient_id-1]
    validation_dice = dice_score(valid_set_gt[index_start:index_end], validation_prediction[index_start:index_end])
    max_haus = np.mean(calculate_haus(valid_set_gt[index_start:index_end], validation_prediction[index_start:index_end]))
    index_start = index_end
    print(f"Patient {patient_id} \tDice score: {validation_dice: .8f} \tMean Haussdorf: {max_haus: 1.8f}")
    dice_scores.append(validation_dice)
    haus_distances.append(max_haus)
    
# Calculate max, min, mean, std
print("\nMax dice score = {} (Validation Set)".format(np.max(dice_scores)))
print("Min dice score = {} (Validation Set)".format(np.amin(dice_scores)))
print("Mean dice score = {} (Validation Set)".format(np.mean(dice_scores)))
print("STD dice score = {} (Validation Set)".format(np.std(dice_scores)))

print("\nMax (mean) haussdorf = {} (Validation Set)".format(np.amax(haus_distances)))
print("Min (mean) haussdorf = {} (Validation Set)".format(np.amin(haus_distances)))
print("Mean (mean) haussdorf = {} (Validation Set)".format(np.mean(haus_distances)))
print("STD (mean) haussdorf = {} (Validation Set)".format(np.std(haus_distances)))

Patient 4 	Dice score:  0.96471998 	Mean Haussdorf:  2.32537731
Patient 12 	Dice score:  0.93983792 	Mean Haussdorf:  2.74052570
Patient 17 	Dice score:  0.95036971 	Mean Haussdorf:  3.15972927
Patient 26 	Dice score:  0.91032763 	Mean Haussdorf:  2.83188605
Patient 30 	Dice score:  0.85494364 	Mean Haussdorf:  2.56674008
Patient 34 	Dice score:  0.83113602 	Mean Haussdorf:  3.27735820
Patient 41 	Dice score:  0.92988609 	Mean Haussdorf:  2.68435291
Patient 42 	Dice score:  0.96372595 	Mean Haussdorf:  2.48580969
Patient 50 	Dice score:  0.95174867 	Mean Haussdorf:  2.51754755
Patient 62 	Dice score:  0.95640998 	Mean Haussdorf:  2.10450426
Patient 69 	Dice score:  0.94620162 	Mean Haussdorf:  2.05096659
Patient 80 	Dice score:  0.96622836 	Mean Haussdorf:  1.76158023
Patient 90 	Dice score:  0.93134637 	Mean Haussdorf:  2.01301952
Patient 91 	Dice score:  0.87301632 	Mean Haussdorf:  2.66281747
Patient 93 	Dice score:  0.94764785 	Mean Haussdorf:  1.83152535

Max dice score = 0.966228

## Ejection Fraction

In [8]:
EF_predictions = []
index = 0
new_scale_factor = 1.0 # the cropping was performed with this number

ef_differences = []
for patient_id in valid_ids:
    ED_prediction = validation_prediction[index:index+z_dim[patient_id-1]]
    ES_prediction = validation_prediction[index+z_dim[patient_id-1]:index+2*z_dim[patient_id-1]]
    index = index + 2*z_dim[patient_id-1]
    EF_prediction, strokevolume = calculate_EF(ED_prediction, ES_prediction, spacing=[1/new_scale_factor,1/new_scale_factor,spacing[patient_id-1][2]])
    diff = (EF_gt[patient_id-1]-EF_prediction)/EF_gt[patient_id-1]*100
    EF_predictions.append(EF_prediction)
    ef_differences.append(diff)
    print(f"Patient {patient_id} \tEF_gt={EF_gt[patient_id-1]:2.4f} \tEF_pred={EF_prediction:2.4f} \tdifference={diff:2.4f}%")
    
# Calculate max, min, mean, std
print("\nMax (abs) EF difference = {}% (Validation Set)".format(np.amax(np.abs(ef_differences))))
print("Min (abs) EF difference = {}% (Validation Set)".format(np.min(np.abs(ef_differences))))
print("Mean EF difference = {}% (Validation Set)".format(np.min(ef_differences)))
print("Std EF difference = {} (Validation Set)".format(np.std(ef_differences)))

Patient 4 	EF_gt=13.1781 	EF_pred=14.1280 	difference=-7.2087%
Patient 12 	EF_gt=24.4246 	EF_pred=19.1120 	difference=21.7508%
Patient 17 	EF_gt=13.6644 	EF_pred=12.8063 	difference=6.2802%
Patient 26 	EF_gt=50.0246 	EF_pred=46.8973 	difference=6.2516%
Patient 30 	EF_gt=74.5888 	EF_pred=70.0245 	difference=6.1193%
Patient 34 	EF_gt=77.2301 	EF_pred=55.0805 	difference=28.6800%
Patient 41 	EF_gt=36.0726 	EF_pred=25.3829 	difference=29.6338%
Patient 42 	EF_gt=33.0754 	EF_pred=32.7564 	difference=0.9645%
Patient 50 	EF_gt=16.4440 	EF_pred=21.0675 	difference=-28.1167%
Patient 62 	EF_gt=56.7987 	EF_pred=52.4708 	difference=7.6198%
Patient 69 	EF_gt=55.4649 	EF_pred=53.4715 	difference=3.5940%
Patient 80 	EF_gt=62.1783 	EF_pred=63.1054 	difference=-1.4911%
Patient 90 	EF_gt=44.8602 	EF_pred=42.2795 	difference=5.7527%
Patient 91 	EF_gt=54.6053 	EF_pred=49.2873 	difference=9.7388%
Patient 93 	EF_gt=58.5547 	EF_pred=60.1358 	difference=-2.7004%

Max (abs) EF difference = 29.633830568129337% (

# Prediction on Training Set

## Predict

In [9]:
# Predict on the training set and save the prediction
if not(os.path.exists("prediction_scaled_crop_train.npy")):
    prediction = loaded_model.predict(train_set, batch_size=10, verbose=1)
    np.save("prediction_scaled_crop_train", prediction)

In [10]:
# Load and preprocess the prediction of the training set
prediction = np.load("prediction_scaled_crop_train.npy")

prediction /= np.amax(prediction)
prediction = prediction[:,:,:,0]

train_set = train_set[:,:,:,0]
train_set_gt  = train_set_gt[:,:,:,0]

## Dice score and Haussdorf distance

In [11]:
# Keep only predictions that equal to 1
train_prediction = copy.deepcopy(prediction)
train_prediction[train_prediction<1] = 0

# Calculate dice score and haussdorf distance for each patient
dice_scores = []
haus_distances = []
index_start = 0
for patient_id in train_ids:
    index_end = index_start + 2*z_dim[patient_id-1]
    train_dice = dice_score(train_set_gt[index_start:index_end], train_prediction[index_start:index_end])
    max_haus = np.mean(calculate_haus(train_set_gt[index_start:index_end], train_prediction[index_start:index_end]))
    index_start = index_end
    print(f"(Train) Patient {patient_id} \tDice score: {train_dice:.8f} \tMean Haussdorf: {max_haus: 1.8f}")
    dice_scores.append(train_dice)
    haus_distances.append(max_haus)

# Calculate max, min, mean, std
print("\nMax dice score = {} (Training Set)".format(np.max(dice_scores)))
print("Min dice score = {} (Training Set)".format(np.amin(dice_scores)))
print("Mean dice score = {} (Training Set)".format(np.mean(dice_scores)))
print("STD dice score = {} (Training Set)".format(np.std(dice_scores)))

print("\nMax (mean) haussdorf = {} (Training Set)".format(np.amax(haus_distances)))
print("Min (mean) haussdorf = {} (Training Set)".format(np.amin(haus_distances)))
print("Mean (mean) haussdorf = {} (Training Set)".format(np.mean(haus_distances)))
print("STD (mean) haussdorf = {} (Training Set)".format(np.std(haus_distances)))

(Train) Patient 2 	Dice score: 0.94102766 	Mean Haussdorf:  2.33234618
(Train) Patient 3 	Dice score: 0.96600216 	Mean Haussdorf:  2.51994227
(Train) Patient 5 	Dice score: 0.95960769 	Mean Haussdorf:  2.50173888
(Train) Patient 6 	Dice score: 0.92905091 	Mean Haussdorf:  3.17081015
(Train) Patient 7 	Dice score: 0.96516370 	Mean Haussdorf:  2.71940744
(Train) Patient 8 	Dice score: 0.96296369 	Mean Haussdorf:  2.55969355
(Train) Patient 9 	Dice score: 0.96139487 	Mean Haussdorf:  2.37474522
(Train) Patient 10 	Dice score: 0.97045797 	Mean Haussdorf:  2.45476633
(Train) Patient 11 	Dice score: 0.95821980 	Mean Haussdorf:  2.41219227
(Train) Patient 14 	Dice score: 0.95451145 	Mean Haussdorf:  2.59107146
(Train) Patient 15 	Dice score: 0.95296511 	Mean Haussdorf:  2.43863553
(Train) Patient 18 	Dice score: 0.96303593 	Mean Haussdorf:  2.38041767
(Train) Patient 19 	Dice score: 0.97788066 	Mean Haussdorf:  2.28587414
(Train) Patient 20 	Dice score: 0.96403219 	Mean Haussdorf:  2.71925534

## Ejection Fraction

In [12]:
EF_predictions = []
index = 0
new_scale_factor = 1.0 # the cropping was performed with this number

ef_differences = []
for patient_id in train_ids:
    ED_prediction = train_prediction[index:index+z_dim[patient_id-1]]
    ES_prediction = train_prediction[index+z_dim[patient_id-1]:index+2*z_dim[patient_id-1]]
    index = index + 2*z_dim[patient_id-1]
    EF_prediction, strokevolume = calculate_EF(ED_prediction, ES_prediction, spacing=[1/new_scale_factor,1/new_scale_factor,spacing[patient_id-1][2]])
    diff = (EF_gt[patient_id-1]-EF_prediction)/EF_gt[patient_id-1]*100
    EF_predictions.append(EF_prediction)
    ef_differences.append(diff)
    print(f"Patient {patient_id} \tEF_gt={EF_gt[patient_id-1]:2.4f} \tEF_pred={EF_prediction:2.4f} \tdifference={diff:2.4f}%")
    
# Calculate max, min, mean, std
print("\nMax (abs) EF difference = {}% (Training Set)".format(np.amax(np.abs(ef_differences))))
print("Min (abs) EF difference = {}% (Training Set)".format(np.min(np.abs(ef_differences))))
print("Mean EF difference = {}% (Training Set)".format(np.min(ef_differences)))
print("Std EF difference = {} (Training Set)".format(np.std(ef_differences)))

Patient 2 	EF_gt=29.1412 	EF_pred=29.0690 	difference=0.2475%
Patient 3 	EF_gt=12.8728 	EF_pred=10.1593 	difference=21.0790%
Patient 5 	EF_gt=22.9378 	EF_pred=22.4808 	difference=1.9922%
Patient 6 	EF_gt=14.3602 	EF_pred=17.9697 	difference=-25.1354%
Patient 7 	EF_gt=10.4767 	EF_pred=11.6517 	difference=-11.2150%
Patient 8 	EF_gt=16.9730 	EF_pred=15.3801 	difference=9.3853%
Patient 9 	EF_gt=14.9753 	EF_pred=13.0632 	difference=12.7685%
Patient 10 	EF_gt=10.7588 	EF_pred=11.2151 	difference=-4.2411%
Patient 11 	EF_gt=12.3083 	EF_pred=13.5182 	difference=-9.8307%
Patient 14 	EF_gt=14.2034 	EF_pred=12.4778 	difference=12.1494%
Patient 15 	EF_gt=25.7340 	EF_pred=32.2253 	difference=-25.2245%
Patient 18 	EF_gt=22.7535 	EF_pred=23.8672 	difference=-4.8948%
Patient 19 	EF_gt=27.4510 	EF_pred=26.6267 	difference=3.0029%
Patient 20 	EF_gt=17.2904 	EF_pred=16.1499 	difference=6.5964%
Patient 21 	EF_gt=67.5592 	EF_pred=64.3301 	difference=4.7798%
Patient 22 	EF_gt=64.9394 	EF_pred=63.0683 	differ